<a href="https://colab.research.google.com/github/windinherhair/fs_co_tr/blob/master/FS_Train_CUDA9_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# deepfakes/faceswap (Original_HighRes Train用)

## **（重要）colabの12時間&90分ルール対策**

1.新しいインスタンスを起動してから12時間経つと全てリセットされる。（GoogleDriveに学習したmodelを退避させる）

2.90分アイドル状態が続くとノートの接続が外れるため、** Chromeのアドオンを使って60分置きにページをリロードする。（[Auto Refresh](https://chrome.google.com/webstore/detail/auto-refresh/ifooldnmmcmlbdennkpdnlnbgbmfalko))**



### 事前準備

**1.[cuDNN](https://developer.nvidia.com/cudnn) 7.41 for CUDA 9.0 (cuDNN Library for Linux)をダウンロードし、GoogleDriveにアップしておく。アーカイブにあります。要登録。バージョン間違える人多いので注意。**※windowsPCの場合拡張子がおかしくなるかも。拡張子をtgzに直してください。

2.自分のPCでExtractまで済ませておく。（ローカルでやった方が効率がいいため）

3.faceswapという名前のフォルダを作り、そこにalignA, alignBフォルダを移動する。

(3.5.convertもcolabでやりたい方はalignments.jsonファイルを含むimgAフォルダもfaceswapフォルダに配置)←OFSは入ってるけどFSはインスコしてない人向け。

4.faceswapフォルダをzipで圧縮し、faceswap.zipファイルをGoogleDriveにアップしておく

5.事前学習modelがある場合は、modelフォルダをGoogleDriveにアップしておく

### **（重要）ランタイムのタイプをGPUに変更する** 

---



In [0]:
# インスタンスが作られてからの日数表示。0.5days=12時間まで使える。

!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'


準備が整ったら、

**ランタイム→すべてのセルを実行**


---





**Google Driveをマウント**（※ここだけ要入力）


  URLをクリックし、そのまま進めたら出てくるコードをコピー、枠にペーストしEnter




In [0]:
from google.colab import drive
drive.mount('/content/drive')

**元からインストールされているCUDA9.2を削除**

In [0]:
!rm -rf /usr/local/cuda*
!apt-get purge nvidia*
!apt-get autoremove
!apt-get autoclean

**CUDA9.0をインストール**

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb -O cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get -o Dpkg::Options::="--force-overwrite" install cuda-9-0 cuda-drivers

Driveからcudnnをコピペして展開

In [0]:
cp drive/'My Drive'/cudnn-9.0-linux-x64-v7.4.1.5.tgz ./

In [0]:
!tar -xf cudnn-9.0-linux-x64-v7.4.1.5.tgz

In [0]:
cp -R cuda/include/* /usr/local/cuda-9.0/include


In [0]:
cp -R cuda/lib64/* /usr/local/cuda-9.0/lib64

CUDAのパスを通す。

In [0]:
!echo 'export PATH=/usr/local/cuda-9.0/bin${PATH:+:${PATH}}' >> ~/.bashrc
!echo 'export LD_LIBRARY_PATH=/usr/local/cuda-9.0/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}' >> ~/.bashrc

In [0]:
!source ~/.bashrc
!ldconfig

dlibのインストール

In [0]:
!apt update
!apt install -y cmake
#!git clone https://github.com/davisking/dlib.git
#!python setup.py install --yes USE_AVX_INSTRUCTIONS --yes DLIB_USE_CUDA
!apt install -y --no-install-recommends cmake && pip install dlib

TFをダウングレード

In [0]:
!pip install tensorflow-gpu==1.11.0

Githubのdeepfakes/faceswapをクローン

In [0]:
!git clone https://github.com/deepfakes/faceswap.git

素材をドライブからコピー

In [0]:
cp drive/'My Drive'/faceswap.zip ./

素材を解凍

In [0]:
!unzip faceswap.zip

faceswapディレクトリに移動

In [0]:
cd faceswap

残りの必要なものをインストール

In [0]:
!pip install -r requirements.txt*



---




>  ここでやっとFaceswap環境が整う。

> extractやconvertもcolab上でやりたい人は、



> ```
!python faceswap.py -h
```


> でコマンドを見れるよ。


---









In [0]:
#!python faceswap.py train -h

In [0]:
# インスタンスが作られてからの日数表示。0.5days=12時間まで使える。

!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'


Train(バッチ32で100回ごとに保存)※設定はお好みで

In [0]:
#!python faceswap.py train -A alignB/ -B alignB/ -m /content/drive/'My Drive'/model/ -s 100 -t OriginalHighRes -bs 32

In [0]:
!python faceswap.py train -A alignA/ -B alignB/ -m /content/drive/'My Drive'/model/ -s 100 -t OriginalHighRes -bs 32

あとはドライブのmodelフォルダをダウンロードして終わりです。（ローカルでconvertした方が効率がいいため。）
# お疲れ様でした~

※ローカルにFS環境がない方で、convertもcolabで済ませたい方向け。

コメントアウト（#）を外して使ってください。


In [0]:
#python faceswap.py convert -i imgA/ -o /merge -al /content/faceswap/imgA/alignments.json -m /content/drive/'My Drive'/model/ -a alignA/ -t OriginalHighRes -c Masked -e 20 -b 30 -M facehull -mh

In [0]:
#オプションはお好みで
#python faceswap.py convert -h